In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
from model.model import Model

model = Model()
model.load()

In [3]:
def encode_df(df):
    le = LabelEncoder()
    for col, col_type in df.dtypes.items():
        if col_type == 'object':
            df[col] = le.fit_transform(df[col])
    return df


def is_female(number):
    return str(number)[2] == '5' or str(number)[2] == '6'


def get_client_df():
    df = open_csv('client')
    df['gender'] = df.apply(lambda row: 0 if is_female(row['birth_number']) else 1, axis=1)
    df['birth_number'] = df.apply(lambda row: row['birth_number'] - 5000 if is_female(row['birth_number']) else row['birth_number'], axis=1)
    return df


In [21]:
def merge_all(model, op='test'):
    def modify_trans(df, op):
        df.loc[df[f'trans_{op}_type'] == 'withdrawal', f'trans_{op}_amount'] *= -1
        df = df.drop([f'trans_{op}_k_symbol'], axis=1)
        df = df.drop([f'trans_{op}_bank', f'trans_{op}_account'], axis=1)
        return df

    account_df = model.datasets['account']
    client_df = model.datasets['client']
    disp_df = model.datasets['disp']
    district_df = model.datasets['district'].rename(columns={'district_code': 'district_id'})
    card_df = model.datasets[op][f'card_{op}']
    loan_df = model.datasets[op][f'loan_{op}']
    trans_df = model.datasets[op][f'trans_{op}']
    trans_df = modify_trans(trans_df, op)

    ownership_df = disp_df.groupby('account_id', as_index=False, group_keys=False).agg({'disp_type': ['count']})
    ownership_df.columns = [f'{col_name}_{agg_name}' if 'id' not in col_name else col_name for col_name, agg_name in ownership_df.columns]
    disp_df = disp_df.merge(ownership_df, on='account_id')
    disp_df = disp_df[disp_df.disp_type == 'OWNER'].drop(['disp_type'], axis=1)

    df = pd.merge(loan_df, disp_df, on='account_id')
    df = pd.merge(df, client_df, on='client_id')
    df = pd.merge(df, district_df, on='district_id')
    df['account_date'] = pd.merge(df, account_df, on='account_id')['account_date']
    df = pd.merge(df, trans_df, on='account_id')
    return df.sort_values(by=[f'trans_{op}_date'])


train_df = merge_all(model, 'train')
train_df.head()
# test_df = merge_all('test')
# print(get_trans_train_df()['trans_amount'])


,loan_id,account_id,loan_train_date,loan_train_amount,loan_train_duration,loan_train_payments,loan_train_status,disp_id,client_id,disp_type_count,...,district_no._of_enterpreneurs_per_1000_inhabitants,district_no._of_commited_crimes_'95,district_no._of_commited_crimes_'96,account_date,trans_id,trans_train_date,trans_train_type,trans_train_operation,trans_train_amount,trans_train_balance
1988,6077,5270,931122,79608,24,3317,1,6367,6367,1,...,97,2166,2325,930113,1548749,930113,credit,credit in cash,800.0,800.0
1030,7284,11265,930915,52788,12,4399,1,13537,13845,1,...,124,1845,1879,930114,3393738,930114,credit,credit in cash,1000.0,1000.0
1989,6077,5270,931122,79608,24,3317,1,6367,6367,1,...,97,2166,2325,930113,1548750,930114,credit,collection from another bank,44749.0,45549.0
1574,7121,10364,931110,21924,36,609,1,12446,12754,2,...,111,3659,3894,930117,3122924,930117,credit,credit in cash,1100.0,1100.0
6977,5754,3834,940928,23052,12,1921,1,4620,4620,2,...,140,18721,18696,930119,1121963,930119,credit,credit in cash,700.0,700.0


In [ ]:
from agg import *
import numpy as np

def create_stuff(df):
    keep_cols = ['loan_id', 'account_id', 'status', 'loan_date', 'account_date', 'loan_amount', 'duration', 'payments', 'gender', 'birth_number', 'disp_type_count', 'district_id', 'no. of inhabitants', 'no. of municipalities with inhabitants < 499 ', 'no. of municipalities with inhabitants 500-1999',
                    'no. of municipalities with inhabitants 2000-9999 ', 'no. of municipalities with inhabitants >10000 ', 'no. of cities ', 'ratio of urban inhabitants ', 'average salary ', 'unemploymant rate \'95 ', 'unemploymant rate \'96 ', 'no. of enterpreneurs per 1000 inhabitants ', 'no. of commited crimes \'95 ', 'no. of commited crimes \'96 ']

    df = df.groupby(keep_cols, as_index=False, group_keys=False).agg({
        'trans_amount': ['mean','min','max','std','last'],
        'balance': ['mean','min','max','std','last'],
        'trans_type': [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit, cov_withdrawal, cov_credit]
    })
    # df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]

    return df

train_df = create_stuff(train_df)


train_df.head(5)
# train_df.dtypes

loan_id account_id status loan_date account_date loan_amount duration  \
                                                                          
0    4959          2      1    940105       930226       80952       24   
1    4961         19     -1    960429       950407       30276       12   
2    4973         67      1    960502       941019      165960       24   
3    4996        132      1    961106       960511       88440       12   
4    5002        173      1    940531       931126      104808       12   

  payments gender birth_number  ...       balance                trans_type  \
                                ...           std     last count_withdrawal   
0     3373      1       450204  ...  12061.705682  27855.2               29   
1     2523      0       390423  ...  15039.248405  15854.0               32   
2     6915      0       440613  ...  20955.646998  23703.8               84   
3     7370      0       450703  ...  21638.258870  78907.6               14   
4     8734      1       391130  ...  11517.175248  28015.4               18   

                                                                      \
  count_credit mean_withdrawal mean_credit std_withdrawal std_credit   
0           22        0.537037    0.407407       0.498626   0.491352   
1           46        0.400000    0.575000       0.489898   0.494343   
2           37        0.672000    0.296000       0.469485   0.456491   
3           16        0.451613    0.516129       0.497653   0.499740   
4           12        0.600000    0.400000       0.489898   0.489898   

                                             
        cov_withdrawal           cov_credit  
0    0.253319357092942   0.2459818308874912  
1   0.2430379746835443  0.24746835443037976  
2  0.22219354838709673   0.2100645161290322  
3  0.25591397849462355   0.2580645161290323  
4  0.24827586206896562  0.24827586206896562  

[5 rows x 43 columns]

In [ ]:
train_df.to_csv('data/train.csv', index=False, sep=';')